In [ ]:
!pip install plotly==5.21.0

In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import graphtools
from scipy.interpolate import Rbf
from sklearn.preprocessing import MinMaxScaler

def compute_distdeg(input_df, split_by="celltype", 
    col_list=["PHATE1","PHATE2"], 
    knn=50, distance_metric = "manhattan", scale = True, inv_degree=True,
    plot=False, rmv_outliers=True):
    #Function that computes distances and degrees of cells within a cluster
    #Results will get scaled within the cluster level so it handles them as independent entities
    output_df = pd.DataFrame(columns=col_list+["dist","deg"])
    for i in input_df[split_by].unique():
        print(i)
        df_dist = input_df.loc[input_df[split_by]==i,col_list]
        #Knn param is key and must be tunned according to the size of the cluster
        G = graphtools.Graph(df_dist.values, knn=knn, decay=None, distance=distance_metric)
        # Return distances by using shortest_path
        np_dist = G.shortest_path(distance="data")
        #Compute median distance to all cells for al cells -> min should be a centroid
        df_dist["dist"] = np.median(np_dist, axis=1)
        df_dist["deg"] = G.to_pygsp().d
        if rmv_outliers:
            #Replace distant outliers with median value -> DONE to smoothen landscape at the very edges of a cluster
            df_dist["dist"] = np.where(
                                df_dist["dist"]>np.percentile(df_dist["dist"],99), 
                                df_dist["dist"].median(), 
                                df_dist["dist"])
        if scale:
            df_dist["dist_scaled"] = MinMaxScaler().fit_transform(X = df_dist[["dist"]])
            df_dist["inv_deg_scaled"] = MinMaxScaler().fit_transform(X = 1/df_dist[["deg"]])
            
        output_df = pd.concat([output_df, df_dist])
        
    return output_df





In [ ]:
# This will not be completed with this simple codes
# Just proceed to the next cell

input_df = pd.read_csv("D:/KP/EKP_RKP/phate/PHATE.Default.output.meta.txt", encoding='unicode_escape', sep = "\t")
output_df = compute_distdeg(input_df)


In [83]:
input_df

,PHATE1,PHATE2,batch,sample_batch,tissue,type,leiden,n_genes,n_genes_by_counts,total_counts,...,velocity_self_transition,root_cells,end_points,velocity_pseudotime,latent_time,velocity_length,velocity_confidence,velocity_confidence_transition,scvelo_leiden,celltype
AAACCCACAGTATGAA,-0.022671,0.001551,0,-1-0,allograft,EKP,0,3835,3835,16059,...,0.035713,1.627170e-01,1.438578e-02,0.336311,0.834071,167.25,0.516420,0.962171,0,Zeb2+ cell_1
AAACGAATCTCGCTTG,-0.012813,0.009862,0,-1-0,allograft,EKP,2,656,656,1903,...,0.010161,6.179012e-01,3.949440e-01,0.520379,0.135188,290.87,0.099673,0.970261,4,Cdk8+ cell_1
AAACGCTAGTGGCGAT,-0.023366,-0.003843,0,-1-0,allograft,EKP,0,4898,4898,26809,...,0.291017,8.648462e-02,5.984261e-03,0.340351,0.862483,109.88,0.625648,0.651285,1,Runx2/Runx3+ cell_1
AAACGCTCATGGAATA,-0.019877,-0.013829,0,-1-0,allograft,EKP,0,4000,4000,21023,...,0.228161,1.273629e-01,1.728421e-05,0.401556,0.949667,100.40,0.813403,0.780288,2,Runx1+ stem_1
AAAGAACGTACTCGCG,-0.020469,0.003017,0,-1-0,allograft,EKP,0,3739,3739,15947,...,0.486793,1.819686e-02,8.216390e-03,0.356933,0.853877,123.90,0.725516,0.513673,0,Zeb2+ cell_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGACTAGGACGGAG,-0.018680,-0.006216,1,-1-1,allograft,RKP,0,3833,3833,20288,...,0.025429,1.144066e-07,1.184737e-03,0.668227,0.403718,40.82,0.978166,0.936110,5,Zeb2/Runx2/Runx3+ cell
TTTGACTGTCGCTGCA,-0.018257,-0.008650,1,-1-1,allograft,RKP,0,4807,4807,27655,...,0.032758,1.623987e-07,5.240720e-05,0.687539,0.507764,35.31,0.983380,0.931862,8,Runx2/Runx3+ cell_2
TTTGATCCATGACTTG,0.002079,0.010600,1,-1-1,allograft,RKP,2,694,694,1473,...,0.650252,1.062666e-03,1.225281e-07,0.644648,0.607413,14.36,0.689613,0.275908,3,Eno1+ cell
TTTGGAGGTCTCGCGA,0.003136,0.009108,1,-1-1,allograft,RKP,2,1346,1346,4412,...,0.569004,2.785794e-03,6.233914e-08,0.657289,0.631854,19.05,0.671735,0.317642,3,Eno1+ cell


In [171]:
df = pd.crosstab(input_df['leiden'], input_df['leiden'])
df

leiden,0,1,2,3,4,5
leiden,,,,,,
0,753,0,0,0,0,0
1,0,577,0,0,0,0
2,0,0,313,0,0,0
3,0,0,0,40,0,0
4,0,0,0,0,15,0
5,0,0,0,0,0,10


In [84]:
input_df['celltype']

AAACCCACAGTATGAA              Zeb2+ cell_1
AAACGAATCTCGCTTG              Cdk8+ cell_1
AAACGCTAGTGGCGAT       Runx2/Runx3+ cell_1
AAACGCTCATGGAATA             Runx1+ stem_1
AAAGAACGTACTCGCG              Zeb2+ cell_1
                             ...          
TTTGACTAGGACGGAG    Zeb2/Runx2/Runx3+ cell
TTTGACTGTCGCTGCA       Runx2/Runx3+ cell_2
TTTGATCCATGACTTG                Eno1+ cell
TTTGGAGGTCTCGCGA                Eno1+ cell
TTTGTTGCAGGATTCT          Ptma/Krt18+ cell
Name: celltype, Length: 1708, dtype: object

# Run this first and modify knn numbers for each cluster

In [172]:
split_by="type"
col_list=["PHATE1","PHATE2"]
distance_metric = "manhattan"

output_df = pd.DataFrame(columns=col_list+["dist","deg"])
for i in input_df[split_by].unique():
    print(i)
    df_dist = input_df.loc[input_df[split_by]==i,col_list]
    knn = 40
    
    G = graphtools.Graph(df_dist.values, knn=knn, decay=None, distance=distance_metric)
    # Return distances by using shortest_path
    np_dist = G.shortest_path(distance="data")
    #Compute median distance to all cells for al cells -> min should be a centroid
    df_dist["dist"] = np.median(np_dist, axis=1)
    df_dist["deg"] = G.to_pygsp().d
    #Replace distant outliers with median value -> DONE to smoothen landscape at the very edges of a cluster
    df_dist["dist"] = np.where(
                df_dist["dist"]>np.percentile(df_dist["dist"],99), 
                df_dist["dist"].median(), 
                df_dist["dist"])
    df_dist["dist_scaled"] = MinMaxScaler().fit_transform(X = df_dist[["dist"]])
    df_dist["inv_deg_scaled"] = MinMaxScaler().fit_transform(X = 1/df_dist[["deg"]])
    output_df = pd.concat([output_df, df_dist])


EKP
RKP


In [169]:
# Define the highest knn number for the cluster that made error in the previous cell
# Try different knn numbers until it succeed

i== 3
knn = 6

split_by="leiden"
col_list=["PHATE1","PHATE2"]
distance_metric = "manhattan"
df_dist = input_df.loc[input_df[split_by]==i,col_list]
G = graphtools.Graph(df_dist.values, knn=knn, decay=None, distance=distance_metric)
# Return distances by using shortest_path
np_dist = G.shortest_path(distance="data")

In [170]:
split_by="leiden"
col_list=["PHATE1","PHATE2"]
distance_metric = "manhattan"

output_df = pd.DataFrame(columns=col_list+["dist","deg"])
for i in input_df[split_by].unique():
    print(i)
    df_dist = input_df.loc[input_df[split_by]==i,col_list]
    #Knn param is key and must be tunned according to the size of the cluster
    knn = 50
    if i == 3:
        knn=6
    
    G = graphtools.Graph(df_dist.values, knn=knn, decay=None, distance=distance_metric)
    # Return distances by using shortest_path
    np_dist = G.shortest_path(distance="data")
    #Compute median distance to all cells for al cells -> min should be a centroid
    df_dist["dist"] = np.median(np_dist, axis=1)
    df_dist["deg"] = G.to_pygsp().d
    #Replace distant outliers with median value -> DONE to smoothen landscape at the very edges of a cluster
    df_dist["dist"] = np.where(
                df_dist["dist"]>np.percentile(df_dist["dist"],99), 
                df_dist["dist"].median(), 
                df_dist["dist"])
    df_dist["dist_scaled"] = MinMaxScaler().fit_transform(X = df_dist[["dist"]])
    df_dist["inv_deg_scaled"] = MinMaxScaler().fit_transform(X = 1/df_dist[["deg"]])
    output_df = pd.concat([output_df, df_dist])


0
2
3


C:\Users\Park_Lab\anaconda3\envs\phate_KP\lib\site-packages\numpy\lib\function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


ValueError: Input contains infinity or a value too large for dtype('float64').

# Finalize

In [173]:
split_by="type"
col_list=["PHATE1","PHATE2"]
distance_metric = "manhattan"

output_df = pd.DataFrame(columns=col_list+["dist","deg"])
for i in input_df[split_by].unique():
    print(i)
    df_dist = input_df.loc[input_df[split_by]==i,col_list]
    #Knn param is key and must be tunned according to the size of the cluster
    knn = 50
    #if i == 'Zeb2+ cell_1':
    #    knn=49
    #if i == 'Cdk82+ cell_1':
    #    knn=6
    G = graphtools.Graph(df_dist.values, knn=knn, decay=None, distance=distance_metric)
    # Return distances by using shortest_path
    np_dist = G.shortest_path(distance="data")
    #Compute median distance to all cells for al cells -> min should be a centroid
    df_dist["dist"] = np.median(np_dist, axis=1)
    df_dist["deg"] = G.to_pygsp().d
    #Replace distant outliers with median value -> DONE to smoothen landscape at the very edges of a cluster
    df_dist["dist"] = np.where(
                df_dist["dist"]>np.percentile(df_dist["dist"],99), 
                df_dist["dist"].median(), 
                df_dist["dist"])
    df_dist["dist_scaled"] = MinMaxScaler().fit_transform(X = df_dist[["dist"]])
    df_dist["inv_deg_scaled"] = MinMaxScaler().fit_transform(X = 1/df_dist[["deg"]])
    output_df = pd.concat([output_df, df_dist])


EKP
RKP


In [175]:
output_file = "D:/KP/EKP_RKP/phate" + "/graphtools.txt"
output_df.to_csv(output_file)